Имеется файл, зашифрованный в соответствии с агоритмом AES, также предоставлены его параметры – Key и IV:
* Key:

```WRGGNuGS9ykJNTvllzUyJ89D7128Up/J/K+GSU07w3mmSGNRNRTBt+PcKC5V7rFsbje0c6g0CvFSFIHAAQeOOMZYMaqDmGTfMJUG5Ir4Uf0oeF5G5u0BMKp4HGpFDHwAhFhJl0Y2ZerhNXPMo8E4dX0VZKhxjAvUuIFmgk4dzcE= ```

* IV:

```hJN7pjqF1t6Oh7MG6eT48DI7Sv/Ca2CO8e/CJcPw367Bem0U52omnq2bkqVzVI1BzYZ9B7LE4G91ddlOzXKJSAxUdCNrlASqoG57GG1MgBJyq48qFaqDxFYBHo8IHEy2e68FSzQ5/5as5S3tf+/FCWl+u80GLl1eqGnDR/BJkmc=```

Эти данные нужны принимающей стороне, для расшифровки контента. Разберемся со значением IV. Интернет говорит нам, что IV –это вектор инициализации, подаваемый на вход в таких режимах шифрования, как CBC, CFB и OFB. Причем для обычного открытого текста можно использовать CBC, CFB или OFB. Для шифрования файлов лучше пользоваться CBC: значительно увеличивается безопасность, при появлении ошибок в хранимых данных почти никогда не бывает сбоев синхронизации.  Как мы выяснили ранее, по сети у нас передается именно видео файл, поэтому логично предположить, что при шифровании видео был использован режим шифрования именно режим сцепления блоков шифротекста (англ. Cipher Block Chaining, CBC) — один из режимов шифрования для симметричного блочного шифра.


Поиск слабых ключей
-------------------
Имеется база данных открытых ключей RSA. Предположим, что исходный ключ симметричного шифрования был зашифрован одним из этих открытых ключей. Чтобы его расшифровать, нам необходимо получить закрытый ключ. Когда в распоряжении есть несколько открытых ключей, можно предположить, что некоторые из них являются слабыми. Это означает, что они имеют общие значения p или q. Узнав разложение n на множители, можно узнать закрытый ключ для данного открытого ключа. Пробуем найти наибольшие общие делители у ключей с помощью алгоритма Евклида. Если НОД каких-либо ключей не равен 1, это означает, что n, соответствующие ключам делятся на НОД. Делим n на НОД и находим искомое разложение.
Найдем слабые ключи:


In [2]:
import rsa
import base64
import math
key_list=[]
for i in range(1, 101):
    path = 'challenge\\'
    path += str(i)
    path += '.pem'
    with open(path, 'rb') as f:
        public_file = f.read()
    pubkey = rsa.PublicKey.load_pkcs1_openssl_pem(public_file)
    key_list.append(pubkey)
    #print(pubkey.n)

for i in range (len(key_list)-1):
    for j in range (i+1,len(key_list)):
        g=math.gcd(key_list[i].n,key_list[j].n)
        if g!=1:
            print(i, j)
            print(g)

6 92
11714776996979588435440066274186746431207255454202665944875994417925671637361979718990753187254963362590523267356636189119746530253241195041886537439669297
8 43
12746114090037338638405558348364753170762024925046410448750084084398232130814610810740410270513256665449277121134270816451924978100353060942631979836542387
28 81
13255021146474500311567608978490890706169542435903223652857277300729054042351345458519493118521383085015716518452176586805584419240894763844777441159376627
33 79
13307692754962545190733618818859249009308397605584526413665519201272482870171338937280605559186730933842250237720203697942551335313430434560173585438909597
57 70
12913750264623462276337425644128711174675813336880465068461018681645080047918835348323621347173714301224881492646558739535118834131362641424915974945262413
59 96
13223199974589313585283471854827363329451685303943205044027865131047695079691108706585803904203301545648101019060082323960067905432343412585098414381558929


Восстановление закрытого ключа
------------------------------
Восстановим закрытый ключ, например, для 6 открытого ключа. Значение закрытого ключа получаем путем нахождения обратного к числу $e$ по модулю $φ(n)$, где $$φ(n) = (p − 1)(q − 1)$$
$$d=e^{-1}(mod φ(n))$$  


In [5]:
weak1=6
weak2=92
e=key_list[weak1].e
print("e:",e)
p=math.gcd(key_list[weak1].n, key_list[weak2].n)
print("p:",p)
n=key_list[weak1].n
print("n:",n)
q=n//p
print("q:",q)
phi=(p-1)*(q-1)
print("phi:",phi)
d=pow(e,-1,phi)
print("d:",d)
mykey = rsa.PrivateKey(n, e, d, p, q)
print(mykey.d)


e: 65537
p: 11714776996979588435440066274186746431207255454202665944875994417925671637361979718990753187254963362590523267356636189119746530253241195041886537439669297
n: 136269636317215868658126726142543242028128679787201513621377420299644359247151157885793577216543689892988935986714087409150506883630386841292060595217129497897100280678153687017820663980404875865314501020301179267627899307057160787226214936662085381326053730017478234531591680965138499420169342895677786825703
q: 11632285988230946246714551486716113190291520068494163099500564698778303325737351498239617583337540511764131867572374744310703041201014109767591825596334999
phi: 136269636317215868658126726142543242028128679787201513621377420299644359247151157885793577216543689892988935986714087409150506883630386841292060595217129474550037295467619004863202903077545254366538978323472134891068782603082197687894997706291314788822179375362343305520658250515567045164864533417314750821408
d: 575959980772217153948168258258456719742918

Расшифровка ключа AES и IV
--------------------------
Сохраняем ключ и вектор инициализации в файлы key.txt и iv.txt соответственно. Теперь попробуем с помощью закрытого ключа расшифровать наш симметричный ключ 